Lets say I am contemplating my choice of stay at either Downtown Toronto, Harbourfront at 10 Capreol Court, Toronto, ON, Canada Or 
at 10 Capreol Court, Toronto, ON, Canada. I like italian food and I am considering the number of resaturants around me as the basis of my choice. I would chose the one which has a greater choice for me. Hence, I will be using Foursquare in my decision making.

In [1]:
import requests 
import pandas as pd
import random 
import numpy as np

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


from IPython.display import Image 
from IPython.core.display import HTML 
    

from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Folium installed')
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  25.20 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.00 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.56 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.27 MB/s
vincent-0.4.4- 100% |###################

In [2]:
CLIENT_ID = 'D5NKPYRJW3AJIBSHPTVSXHBKKA12ZJ0XG3YDAAWOI43DQMUQ' 
CLIENT_SECRET = '3HNFANGIDWUV2PUQN5PUQ0NLGWR5C4LZM1S4QFYAZ15ZO1IO' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D5NKPYRJW3AJIBSHPTVSXHBKKA12ZJ0XG3YDAAWOI43DQMUQ
CLIENT_SECRET:3HNFANGIDWUV2PUQN5PUQ0NLGWR5C4LZM1S4QFYAZ15ZO1IO


In [3]:
address = '10 Capreol Court, Toronto, ON, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6403625 -79.3960908


In [15]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [16]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=D5NKPYRJW3AJIBSHPTVSXHBKKA12ZJ0XG3YDAAWOI43DQMUQ&client_secret=3HNFANGIDWUV2PUQN5PUQ0NLGWR5C4LZM1S4QFYAZ15ZO1IO&ll=43.6403625,-79.3960908&v=20180604&query=Italian&radius=500&limit=30'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c4ca7bbdb04f57ab42fba3e'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'primary': True,
      'shortName': 'Italian'}],
    'hasPerk': False,
    'id': '4fdf5390e4b0d263e8a3bcd6',
    'location': {'cc': 'CA',
     'country': 'Canada',
     'distance': 552,
     'formattedAddress': ['Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.644645,
       'lng': -79.399556}],
     'lat': 43.644645,
     'lng': -79.399556},
    'name': 'Gourment italian Sandwichs',
    'referralId': 'v-1548527547'}]}}

In [17]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.cc,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,name,referralId
0,"[{'pluralName': 'Italian Restaurants', 'icon':...",False,4fdf5390e4b0d263e8a3bcd6,CA,Canada,552,[Canada],"[{'lat': 43.644645, 'label': 'display', 'lng':...",43.644645,-79.399556,Gourment italian Sandwichs,v-1548527547


In [18]:

filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered


,name,categories,cc,country,distance,formattedAddress,labeledLatLngs,lat,lng,id
0,Gourment italian Sandwichs,Italian Restaurant,CA,Canada,552,[Canada],"[{'lat': 43.644645, 'label': 'display', 'lng':...",43.644645,-79.399556,4fdf5390e4b0d263e8a3bcd6


In [19]:
dataframe_filtered.name

0    Gourment italian Sandwichs
Name: name, dtype: object

In [20]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 


folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


venues_map

In [ ]:
Lets try another neighborhood at 10 York St, Toronto, ON

In [21]:
address = '10 York St, Toronto, ON, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6424814 -79.3813058


In [22]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [23]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=D5NKPYRJW3AJIBSHPTVSXHBKKA12ZJ0XG3YDAAWOI43DQMUQ&client_secret=3HNFANGIDWUV2PUQN5PUQ0NLGWR5C4LZM1S4QFYAZ15ZO1IO&ll=43.6424814,-79.3813058&v=20180604&query=Italian&radius=500&limit=30'

In [27]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5c4caa7bdb04f57ab194f4fe'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'primary': True,
      'shortName': 'Italian'}],
    'hasPerk': False,
    'id': '5b897e92db1d81002c91df8c',
    'location': {'address': '66 Wellington St W',
     'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'distance': 521,
     'formattedAddress': ['66 Wellington St W',
      'Toronto ON M5K 1E7',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.647161,
       'lng': -79.381691}],
     'lat': 43.647161,
     'lng': -79.381691,
     'postalCode': 'M5K 1E7',
     'state': 'ON'},
    'name': 'Fabbrica Rustic Italian',
    'referralId': 'v-1548528251'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi

In [29]:
venues = results['response']['venues']
dataframe2 = json_normalize(venues)
dataframe2.head()




filtered_columns2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe_2 = dataframe2.loc[:, filtered_columns2]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_2['categories'] = dataframe_2.apply(get_category_type, axis=1)


dataframe_2.columns = [column.split('.')[-1] for column in dataframe_2.columns]

dataframe_2

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Fabbrica Rustic Italian,Italian Restaurant,66 Wellington St W,CA,Toronto,Canada,NaN,521,"[66 Wellington St W, Toronto ON M5K 1E7, Canada]","[{'lat': 43.647161, 'label': 'display', 'lng':...",43.647161,-79.381691,M5K 1E7,ON,5b897e92db1d81002c91df8c
1,Joe Badali's Ristorante Italiano & Bar,Italian Restaurant,156 Front St W,CA,Toronto,Canada,at Simcoe St,420,"[156 Front St W (at Simcoe St), Toronto ON M5J...","[{'lat': 43.645129694026686, 'label': 'display...",43.645130,-79.385025,M5J 2L6,ON,4aa9328df964a5206b5220e3
2,Sauté Rosé,Italian Restaurant,55 University Ave.,CA,Toronto,Canada,in Markel Building,502,"[55 University Ave. (in Markel Building), Toro...","[{'lat': 43.646547314182634, 'label': 'display...",43.646547,-79.384029,M5J 2H7,ON,4b7d777df964a52043c02fe3


In [30]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


for lat, lng, label in zip(dataframe_2.lat, dataframe_2.lng, dataframe_2.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


venues_map

Clearly, I would be staying in 10 York St, Toronto, ON, Canada as it has more choice among italian restaurants.